In [1]:
import numpy as np
from numpy.testing import assert_allclose
from astropy.modeling.models import *
from astropy.modeling.models import math as astmath
from gwcs.wcstools import grid_from_bounding_box

from jwst import datamodels
from jwst.transforms import models

In [2]:
tspec = datamodels.open('../../jwreftools/test.asdf')

# make sure the grismconf and jwreftools use the same instrument configuration
tspec.meta.instrument.instance

{'detector': 'NIS', 'filter': 'GR150R', 'name': 'NIRISS', 'pupil': 'F200W'}

In [3]:
tspec.fwcpos_ref = 354.222229


In [4]:
# GRISMCONF Example 1
# pwd
# Out[1]: '/Users/dencheva/dev/GRISM_NIRISS/V3'

# wavelength = DISPL(order,x0,y0,t)
# dx = DISPX(order,x0,y0,t)
# dy = DISPY(order,x0,y0,t)

import grismconf

# Load the Grism Configuration file
C = grismconf.Config("/Users/dencheva/dev/GRISM_NIRISS/V3/NIRISS_F200W_GR150r.conf")
x0 = 500.5
y0 = 600.1
order='+1'

t = np.linspace(0, 1, 10)
wavelength = C.DISPL(order,x0,y0,t)
print(wavelength)
grismconf_dx = C.DISPX(order,x0,y0,t)
grismconf_dy = C.DISPY(order,x0,y0,t)
print('grismconf_dx: ', grismconf_dx)



[0.75       0.92222222 1.09444444 1.26666667 1.43888889 1.61111111
 1.78333333 1.95555556 2.12777778 2.3       ]
grismconf_dx:  [-1.1667485  -1.19505459 -1.22342102 -1.25184778 -1.28033487 -1.30888229
 -1.33749005 -1.36615813 -1.39488654 -1.42367529]


In [5]:
# pipeline
tspec=datamodels.open('/Users/dencheva/dev/jwreftools/test.asdf')
#print(tspec.orders)
# order =1 is tspec.orders[0]
xmodels=tspec.dispx[0]
ymodels=tspec.dispy[0]
pipe_dx = xmodels[0](x0, y0) + t * xmodels[1](x0, y0) + t**2 * xmodels[2](x0, y0)
pipe_dy = ymodels[0](x0, y0) + t * ymodels[1](x0, y0) + t**2 * ymodels[2](x0, y0)
print('pipe_dx :', pipe_dx)
res = assert_allclose(pipe_dx, grismconf_dx)
if res is None:
    res = True
print("grismconf_dx == pipe_dx: ", res)

res = assert_allclose(pipe_dy, grismconf_dy)
if res is None:
    res = True
print("grismconf_dy == pipe_dy: ", res)



pipe_dx : [-1.1667485  -1.19505459 -1.22342102 -1.25184778 -1.28033487 -1.30888229
 -1.33749005 -1.36615813 -1.39488654 -1.42367529]
grismconf_dx == pipe_dx:  True
grismconf_dy == pipe_dy:  True


In [6]:
# The same test with the file from Feb 24th
sspec=datamodels.open('/Users/dencheva/dev/jwst_data/swara-wfss/NIRISS_WFSS_Reffiles_24feb2022/jwst_niriss_specwcs_0012_v3.asdf')

xmodels=sspec.dispx[0]
ymodels=sspec.dispy[0]
pipe_dx = xmodels[0](x0, y0) + t * xmodels[1](x0, y0) + t**2 * xmodels[2](x0, y0)
pipe_dy = ymodels[0](x0, y0) + t * ymodels[1](x0, y0) + t**2 * ymodels[2](x0, y0)
#print('pipe_dx :', pipe_dx)
res = assert_allclose(pipe_dx, grismconf_dx)
if res is None:
    res = True
print("grismconf_dx == pipe_dx: ", res)

res = assert_allclose(pipe_dy, grismconf_dy)
if res is None:
    res = True
print("grismconf_dy == pipe_dy: ", res)


AssertionError: 
Not equal to tolerance rtol=1e-07, atol=0

Mismatched elements: 10 / 10 (100%)
Max absolute difference: 0.03518716
Max relative difference: 0.02471572
 x: array([-1.174995, -1.198493, -1.222047, -1.245657, -1.269322, -1.293043,
       -1.316821, -1.340654, -1.364543, -1.388488])
 y: array([-1.166748, -1.195055, -1.223421, -1.251848, -1.280335, -1.308882,
       -1.33749 , -1.366158, -1.394887, -1.423675])